This notebook is a driver program for _taxes_ library

In [152]:
%load_ext autoreload
%autoreload 2

from taxes.download import get_sheet_links_names, download_sheet_series, get_gus_stats
from taxes.preprocessing import dload_to_df_list
import pandas as pd
import pathlib
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


W tej komórce jest już zastosowana funkcja, która łączy wszystkie te, które są poniżej.

In [156]:
years = [2019, 2020]
df_dict = dload_to_df_list(years)

Beginning download...

20200214_Gminy_za_2019.xlsx downloaded!


20200214_Powiaty_za_2019.xlsx downloaded!


20200214_Miasta_NPP_za_2019.xlsx downloaded!


20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx downloaded!


20200214_Wojewodztwa_za_2019.xlsx downloaded!


All sheets downloaded!

20210215_Gminy_2_za_2020.xlsx downloaded!


20210211_Powiaty_za_2020.xlsx downloaded!


20210215_Miasta_NPP_2_za_2020.xlsx downloaded!


20210211_Metropolia_2020.xlsx downloaded!


20210211_Wojewodztwa_za_2020.xlsx downloaded!


All sheets downloaded!

ludnosc.stan_i_struktura_31.12.2020.zip downloaded!


Download ended.

Beginning preprocessing...

Ended preprocessing!


**Komórki poniżej znikną w kolejnym commicie**

W tej linijce pobieramy wszystkie niezbędne arkusze `.xslx` ze strony rządowej, lub z mirrora jeśli strona rządowa odmówi współpracy

In [139]:
# getting all sheet links 
years = [2019, 2020]
verb = True
gov_dir = {}
for year in years:
    sheets = get_sheet_links_names(year, verb) 
    gov_dir[year] = sheets
    dir_sheets = download_sheet_series(sheets, verb)

gus_dir = get_gus_stats()

print("PIT earnings dir path = {}\nGUS data dir path = {}".format(dir_sheets, gus_dir))

(['https://www.gov.pl/attachment/6594af94-cd1e-49fb-9149-99fd663aef25', 'https://www.gov.pl/attachment/5f2abc44-6a7e-4b73-8999-696920252efc', 'https://www.gov.pl/attachment/141da745-800d-44c5-ac97-e90c4cbd5e11', 'https://www.gov.pl/attachment/12150aff-d70e-412b-afdc-2bc5341dc823', 'https://www.gov.pl/attachment/141eeb3c-dedc-4491-b0bf-895587824eff'], ['20200214_Gminy_za_2019.xlsx', '20200214_Powiaty_za_2019.xlsx', '20200214_Miasta_NPP_za_2019.xlsx', '20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx', '20200214_Wojewodztwa_za_2019.xlsx']) 5
20200214_Gminy_za_2019.xlsx downloaded!


20200214_Powiaty_za_2019.xlsx downloaded!


20200214_Miasta_NPP_za_2019.xlsx downloaded!


20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx downloaded!


20200214_Wojewodztwa_za_2019.xlsx downloaded!


All sheets downloaded!

(['https://www.gov.pl/attachment/31d60032-a3c5-4e4f-8af8-67c8fa09afd2', 'https://www.gov.pl/attachment/82cb06d7-02e6-4d24-a8b4-9926fe0a3079', 'https://www.gov.pl/attachment/bafb6020-bc

In [141]:
print(gov_dir)

{2019: (['https://www.gov.pl/attachment/6594af94-cd1e-49fb-9149-99fd663aef25', 'https://www.gov.pl/attachment/5f2abc44-6a7e-4b73-8999-696920252efc', 'https://www.gov.pl/attachment/141da745-800d-44c5-ac97-e90c4cbd5e11', 'https://www.gov.pl/attachment/12150aff-d70e-412b-afdc-2bc5341dc823', 'https://www.gov.pl/attachment/141eeb3c-dedc-4491-b0bf-895587824eff'], ['20200214_Gminy_za_2019.xlsx', '20200214_Powiaty_za_2019.xlsx', '20200214_Miasta_NPP_za_2019.xlsx', '20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx', '20200214_Wojewodztwa_za_2019.xlsx']), 2020: (['https://www.gov.pl/attachment/31d60032-a3c5-4e4f-8af8-67c8fa09afd2', 'https://www.gov.pl/attachment/82cb06d7-02e6-4d24-a8b4-9926fe0a3079', 'https://www.gov.pl/attachment/bafb6020-bca0-4ec8-9369-845e0afb94d9', 'https://www.gov.pl/attachment/e4077a76-1fbc-478e-a15d-eea0a4e3f130', 'https://www.gov.pl/attachment/0b98f8be-e9e1-48e3-8bc5-796e8c0b169e'], ['20210215_Gminy_2_za_2020.xlsx', '20210211_Powiaty_za_2020.xlsx', '20210215_Miasta_NPP_

In [109]:
names = {}
for year in gov_dir.keys():
    names[year] = gov_dir[year][1]
names

{2019: ['20200214_Gminy_za_2019.xlsx',
  '20200214_Powiaty_za_2019.xlsx',
  '20200214_Miasta_NPP_za_2019.xlsx',
  '20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx',
  '20200214_Wojewodztwa_za_2019.xlsx'],
 2020: ['20210215_Gminy_2_za_2020.xlsx',
  '20210211_Powiaty_za_2020.xlsx',
  '20210215_Miasta_NPP_2_za_2020.xlsx',
  '20210211_Metropolia_2020.xlsx',
  '20210211_Wojewodztwa_za_2020.xlsx']}

To będie funkcją, która przyporządkowuje nazwę pliku typowi JST jaki opisuje.
Uwaga! W pliku `Miasta_NPP` połowa rekordów jest dla cechy `powiat` w miastach, więc trzeba ją wywalić do df z powiatami.

In [112]:
for year in names.keys():
    classes = dict.fromkeys(["Gminy","Powiaty","Miasta_NPP","Metropolia","Wojewodztwa"])
    for key in classes.keys():
        for name in names[year]:
            if re.search(key, name) is not None:
                classes[key] = name
            else:
                continue
    names[year] = classes

In [146]:
print(names)

{2019: {'Gminy': '20200214_Gminy_za_2019.xlsx', 'Powiaty': '20200214_Powiaty_za_2019.xlsx', 'Miasta_NPP': '20200214_Miasta_NPP_za_2019.xlsx', 'Metropolia': '20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx', 'Wojewodztwa': '20200214_Wojewodztwa_za_2019.xlsx'}, 2020: {'Gminy': '20210215_Gminy_2_za_2020.xlsx', 'Powiaty': '20210211_Powiaty_za_2020.xlsx', 'Miasta_NPP': '20210215_Miasta_NPP_2_za_2020.xlsx', 'Metropolia': '20210211_Metropolia_2020.xlsx', 'Wojewodztwa': '20210211_Wojewodztwa_za_2020.xlsx'}}


In [138]:
df_dict = dict.fromkeys(names.keys())

for year in df_dict.keys():
    df_dict[year] = dict.fromkeys(names[year].keys())
    for key in df_dict[year].keys():
        fl = dir_sheets.joinpath(names[year][key])
        print(fl)

        col_names = ["WK", "PK", "GK", "GT", "Nazwa JST", "województwo", "powiat", "DZIAŁ", "ROZDZIAŁ", "PARAGRAF", 
                     "Należności (saldo początkowe plus przypisy minus odpisy)", "Dochody wykonane (wpłaty minus zwroty)", 
                     "ogółem", "zaległości netto", "nadpłaty"]
        
        file_extension = fl.suffix.lower()[1:]

        if file_extension == 'xlsx':
            df = pd.read_excel(fl, skiprows=7, header=None, names=col_names, engine='openpyxl')
        elif file_extension == 'xls':
            df = pd.read_excel(fl, skiprows=7, header=None, names=col_names, engine='xlrd')
        else:
            raise Exception("File not supported")

        df_dict[year][key] = df
    
df_dict[2020]["Gminy"]

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Gminy_za_2019.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Powiaty_za_2019.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Miasta_NPP_za_2019.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Wojewodztwa_za_2019.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20210215_Gminy_2_za_2020.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20210211_Powiaty_za_2020.xlsx
/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/da

,WK,PK,GK,GT,Nazwa JST,województwo,powiat,DZIAŁ,ROZDZIAŁ,PARAGRAF,Należności (saldo początkowe plus przypisy minus odpisy),Dochody wykonane (wpłaty minus zwroty),ogółem,zaległości netto,nadpłaty
0,2,1,1,1,BOLESŁAWIEC,dolnośląskie,bolesławiecki,756,75621,10,40456699,40456699,NaN,NaN,NaN
1,2,1,2,2,BOLESŁAWIEC,dolnośląskie,bolesławiecki,756,75621,10,13789712,13789712,NaN,NaN,NaN
2,2,1,3,2,GROMADKA,dolnośląskie,bolesławiecki,756,75621,10,3573540,3573540,NaN,NaN,NaN
3,2,1,4,3,NOWOGRODZIEC,dolnośląskie,bolesławiecki,756,75621,10,9553744,9553744,NaN,NaN,NaN
4,2,1,5,2,OSIECZNICA,dolnośląskie,bolesławiecki,756,75621,10,5844560,5844560,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406,32,18,1,3,DOBRA,zachodniopomorskie,łobeski,756,75621,10,2010454,2010454,NaN,NaN,NaN
2407,32,18,2,3,ŁOBEZ,zachodniopomorskie,łobeski,756,75621,10,8834999,8834999,NaN,NaN,NaN
2408,32,18,3,2,RADOWO MAŁE,zachodniopomorskie,łobeski,756,75621,10,1546533,1546533,NaN,NaN,NaN
2409,32,18,4,3,RESKO,zachodniopomorskie,łobeski,756,75621,10,6612100,6612100,NaN,NaN,NaN


In [145]:
type(df_dict[2020]["Gminy"])

pandas.core.frame.DataFrame

In [ ]:
# df_1 = df[df["Nazwa JST"] == "BOLESŁAWIEC"]
# df_1 = df_1[df_1["województwo"] == "dolnośląskie"]
# df_1

#### Pre-commit routine

In [157]:
!pytest 
!pre-commit run --all --show-diff-on-failure

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-7.0.0, pluggy-1.0.0
rootdir: /Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final
plugins: anyio-3.5.0
collected 8 items                                                              

taxes/tests/test_download.py .....                                       [ 62%]
taxes/tests/test_preprocessing.py ...                                    [100%]

=============================== warnings summary ===============================
../../../../../../miniforge3/lib/python3.9/site-packages/_pytest/config/__init__.py:716
  /Users/k4cp3rskiii/miniforge3/lib/python3.9/site-packages/_pytest/config/__init__.py:716: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: taxes.download
    self.import_plugin(import_spec)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 8 passed,